In [19]:
import httpx
import mlcroissant as mlc
import pandas as pd
from pathlib import Path

In [13]:
# CONSTANTS
PARENT_FOLDER = Path.cwd()
DATA_FOLDER = PARENT_FOLDER / "data"
DATASET_URL = (
    "https://www.kaggle.com/datasets/CooperUnion/cardataset/croissant/download"
)

In [21]:
def download_data(url: str):
    dataset = mlc.Dataset(url)

    return dataset

car_dataset = download_data(DATASET_URL)
car_record_sets = car_dataset.metadata.record_sets
dataset_uuid = car_record_sets[0].uuid
car_records = car_dataset.records(dataset_uuid)

df_car_raw = pd.DataFrame(car_records)
df_car_raw.head()

  -  [Metadata(Car Features and MSRP)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.


,data.csv/Make,data.csv/Model,data.csv/Year,data.csv/Engine+Fuel+Type,data.csv/Engine+HP,data.csv/Engine+Cylinders,data.csv/Transmission+Type,data.csv/Driven_Wheels,data.csv/Number+of+Doors,data.csv/Market+Category,data.csv/Vehicle+Size,data.csv/Vehicle+Style,data.csv/highway+MPG,data.csv/city+mpg,data.csv/Popularity,data.csv/MSRP
0,b'BMW',b'1 Series M',2011,b'premium unleaded (required)',335.0,6.0,b'MANUAL',b'rear wheel drive',2.0,"b'Factory Tuner,Luxury,High-Performance'",b'Compact',b'Coupe',26,19,3916,46135
1,b'BMW',b'1 Series',2011,b'premium unleaded (required)',300.0,6.0,b'MANUAL',b'rear wheel drive',2.0,"b'Luxury,Performance'",b'Compact',b'Convertible',28,19,3916,40650
2,b'BMW',b'1 Series',2011,b'premium unleaded (required)',300.0,6.0,b'MANUAL',b'rear wheel drive',2.0,"b'Luxury,High-Performance'",b'Compact',b'Coupe',28,20,3916,36350
3,b'BMW',b'1 Series',2011,b'premium unleaded (required)',230.0,6.0,b'MANUAL',b'rear wheel drive',2.0,"b'Luxury,Performance'",b'Compact',b'Coupe',28,18,3916,29450
4,b'BMW',b'1 Series',2011,b'premium unleaded (required)',230.0,6.0,b'MANUAL',b'rear wheel drive',2.0,b'Luxury',b'Compact',b'Convertible',28,18,3916,34500
